In [1]:
import pickle
import pandas as pd
import csv
import json
import os
import io
from stanza.utils.conll import CoNLL
from stanza.models.common.doc import Document
from sklearn.model_selection import train_test_split


In [2]:
#FINAL RESULT DOC key:dream_id, value:stanza_doc
final_data={}

# APPEN PART

In [3]:
#Load data used for dataset creation
df_shuffled_old = pd.read_pickle('intermediate_store/preprocessed_ads_V1.0.pickle')

#Load reviewed annotation data Appen
with open('reviewed_data/initial_1300.json') as json_file:
    reviewed_appen = json.load(json_file)

df_shuffled_old["new_doc"]=""
    

In [4]:
def split_with_indexes_appen(string):
    token_list = string.split(' ')
    tuple_list = []
    curr_start = 0
    for token in token_list:
        tuple_list.append( (token, curr_start, curr_start+len(token)-1) )#real end char, not start of next one
        curr_start += len(token) + 1
    return tuple_list
def annotate_doc_appen(review_json, doc):
    
    #create splitted review token list
    #create a tag list with same length
    reviewed_tuples = split_with_indexes_appen(review_json['data']['clean_description'])
    tag_list_rev = ['O' for i in range(len(reviewed_tuples))]
    
    #For each annotation check which tokens shall be labelled
    for tag_dict in review_json['completions'][0]['result']:
        first=True
        
        for j, (token, r_start, r_end) in enumerate(reviewed_tuples):
            tag_start = tag_dict['value']['start']
            tag_end = tag_dict['value']['end'] - 1 #real end char, not start of next one
            #print('r_start {} r_end {} tag_start {} tag_end {} token: {}'.format(r_start, r_end, tag_start, tag_end, token))
            if tag_end >= r_start and tag_start <= r_end and tag_dict['value']['labels'] == ['Drug']:
                if first:
                    tag_list_rev[j] = 'B-Drug'
                    first=False
                else:
                    tag_list_rev[j] = 'I-Drug'
    
    #loop over doc and annotate the tokens in misc.
    token_id=0
    for sentence in doc:
        for token_dict in sentence:
            token_dict['misc'] = token_dict['misc'] + '|label=' + tag_list_rev[token_id]
            token_id+=1
    return doc

In [5]:
amt_same_len = 0
amt_not_same_len = 0
amt_exactly_same = 0
amt_not_exactly_same = 0

for i, review in enumerate(reviewed_appen):
    p_id = review['data']['p_id']
    flat_list_orig = [item for sublist in df_shuffled_old.loc[p_id].doc.to_dict() for item in sublist]
    flat_list_rev =review['data']['clean_description'].split(' ')
    length_problem=False
    
    #Check if length matches
    if len(flat_list_orig) == len(flat_list_rev):
        amt_same_len+=1
    else:
        amt_not_same_len +=1
        print('Length Problem at pandas_id: {} and json item nr: {}'.format(p_id, i))
        length_problem=True
    
    #Check each token if they are the equal:
    same = True
    for j, rev_span in enumerate(flat_list_rev):
        if not flat_list_orig[j]['text']==rev_span:
            if not ( flat_list_orig[j]['text'] == rev_span[:-1] and rev_span[-1:] == '.' ):
                same = False
                print('Detail Problem at pandas_id: {} and json item nr: {}'.format(p_id, i))
                break
    if same:
        amt_exactly_same+=1
        #If length and exact tokens are ok --> add label!
        if not length_problem:
            new_doc = annotate_doc_appen(review, df_shuffled_old.loc[p_id].doc.to_dict())
            new_doc = Document(new_doc)
            df_shuffled_old.at[p_id, 'new_doc'] = new_doc
            final_data[review['data']['dream_id']]=new_doc
    else:
        amt_not_exactly_same+=1
        
print('{} rows had still the same amount of tokens. {} not. this is {} %'.format(amt_same_len, amt_not_same_len, amt_not_same_len/amt_same_len ))
print('{} rows had exactly the same tokens. {} not. this is {} %'.format(amt_exactly_same, amt_not_exactly_same, amt_not_exactly_same/amt_exactly_same))
print('%d docs are in the final result'%(len(final_data.keys())))

Length Problem at pandas_id: 742 and json item nr: 344
Detail Problem at pandas_id: 742 and json item nr: 344
Length Problem at pandas_id: 802 and json item nr: 1105
1307 rows had still the same amount of tokens. 2 not. this is 0.001530221882172915 %
1308 rows had exactly the same tokens. 1 not. this is 0.0007645259938837921 %
1016 docs are in the final result


In [6]:
df_shuffled_old = df_shuffled_old[df_shuffled_old['new_doc'] != ""]
df_shuffled_old.shape

(1016, 8)

# Amazon MTurk Part

In [7]:
df_shuffled_new = pd.read_pickle('intermediate_store/preprocessed_AMT_ads_V1.2.pickle')

df_shuffled_new["new_doc"]=""

filename_amt='reviewed_data/AMT_Reviewed_Data_270321.json'

#Load reviewed annotation data
with open(filename_amt) as json_file:
    reviewed_amt = json.load(json_file)

In [8]:
df_shuffled_new['clean_description'] = df_shuffled_new['clean_description'].str.replace('&#44', ',')
df_shuffled_new['clean_description'] = df_shuffled_new['clean_description'].str.replace('&#39', '\'')
df_shuffled_new['clean_description'] = df_shuffled_new['clean_description'].str.replace('&#34', '\"')

In [9]:
# Reverse HTML encoding of characters, which was necessary for AMT platform
def replace_char_in_doc(char, replacement, doc):
    new_doc=doc.to_dict()
    for i, sentence in enumerate(new_doc):
        for j, token in enumerate(sentence):
            #print('i:%d, j:%d'%(i, j))
            new_doc[i][j]['text'] = token['text'].replace(char, replacement)
        
    return Document(new_doc)
 
df_shuffled_new['doc'] = df_shuffled_new['doc'].apply(lambda doc: replace_char_in_doc('&#44', ',', doc))

In [10]:
def split_with_indexes_amt(string):
    token_list = string.split(' ')
    tuple_list = []
    curr_start = 0
    for token in token_list:
        tuple_list.append( (token, curr_start, curr_start+len(token)-1) )#real end char, not start of next one
        curr_start += len(token) + 1
    return tuple_list
def annotate_doc_amt(review_json, doc):
    
    #create splitted review token list
    #create a tag list with same length
    reviewed_tuples = split_with_indexes_amt(review_json['data']['clean_description'])
    tag_list_rev = ['O' for i in range(len(reviewed_tuples))]
    
    #For each annotation check which tokens shall be labelled
    for tag_dict in review_json['completions'][0]['result']:
        first=True
        
        for j, (token, r_start, r_end) in enumerate(reviewed_tuples):
            tag_start = tag_dict['value']['start']
            tag_end = tag_dict['value']['end'] - 1 #real end char, not start of next one
            #print('r_start {} r_end {} tag_start {} tag_end {} token: {}'.format(r_start, r_end, tag_start, tag_end, token))
            if tag_end >= r_start and tag_start <= r_end and tag_dict['value']['labels'] == ['Drug']:
                if first:
                    tag_list_rev[j] = 'B-Drug'
                    first=False
                else:
                    tag_list_rev[j] = 'I-Drug'
    
    #loop over doc and annotate the tokens in misc.
    token_id=0
    for sentence in doc:
        for token_dict in sentence:
            token_dict['misc'] = token_dict['misc'] + '|label=' + tag_list_rev[token_id]
            token_id+=1
    return doc
    

In [11]:
amt_same_len = 0
amt_not_same_len = 0
amt_exactly_same = 0
amt_not_exactly_same = 0

for i, review in enumerate(reviewed_amt):
    p_id = review['data']['p_id']
    flat_list_orig = [item for sublist in df_shuffled_new.loc[p_id].doc.to_dict() for item in sublist]
    flat_list_rev =review['data']['clean_description'].split(' ')
    length_problem=False
    
    #Check if length matches
    if len(flat_list_orig) == len(flat_list_rev):
        amt_same_len+=1
    else:
        amt_not_same_len +=1
        print('Length Problem at pandas_id: {} and json item nr: {}'.format(p_id, i))
        length_problem=True
    
    #Check each token if they are the equal:
    same = True
    for j, rev_span in enumerate(flat_list_rev):
        if not flat_list_orig[j]['text']==rev_span:
            if not ( flat_list_orig[j]['text'] == rev_span[:-1] and rev_span[-1:] == '.' ):
                same = False
                print('Detail Problem at pandas_id: {} and json item nr: {}'.format(p_id, i))
                print('Orig_flat_list: {} and rev_span: {}'.format(flat_list_orig[j]['text'], rev_span))
                break
    if same:
        amt_exactly_same+=1
        #If length and exact tokens are ok --> add label!
        if not length_problem:
            new_doc = annotate_doc_amt(review, df_shuffled_new.loc[p_id].doc.to_dict())
            new_doc = Document(new_doc)
            df_shuffled_new.at[p_id, 'new_doc'] = new_doc
            final_data[review['data']['dream_id']]=new_doc
    else:
        amt_not_exactly_same+=1

if amt_not_same_len > 0:
    print('{} rows had still the same amount of tokens. {} not. this is {} %'.format(amt_same_len, amt_not_same_len,
                                                                                     amt_not_same_len/amt_same_len ))
else:
    print('{} rows had still the same amount of tokens. {} not. this is {} %'.format(amt_same_len, amt_not_same_len, 1 ))

if amt_not_exactly_same > 0:
    print('{} rows had exactly the same tokens. {} not. this is {} %'.format(amt_exactly_same, amt_not_exactly_same,
                                                                         amt_not_exactly_same/amt_exactly_same))
else:
    print('{} rows had exactly the same tokens. {} not. this is {} %'.format(amt_exactly_same, amt_not_exactly_same, 1))
    
print('%d docs are in the final result'%(len(final_data.keys())))

Length Problem at pandas_id: 384 and json item nr: 415
Detail Problem at pandas_id: 384 and json item nr: 415
Orig_flat_list: - - and rev_span: -
Length Problem at pandas_id: 883 and json item nr: 916
Detail Problem at pandas_id: 883 and json item nr: 916
Orig_flat_list:  and rev_span: Grandaddy
Length Problem at pandas_id: 2188 and json item nr: 1007
Detail Problem at pandas_id: 2188 and json item nr: 1007
Orig_flat_list: q0qLeBxj553tAM2cjZCqxPWIVJwunVDEdpzlUroFXXmgOI7A9myk9CqagqNFw1i 2TXfe5nLlinNdxEKNCeQehhWeryPY5/WC5PRABEBAAG0GkNhbmphbTQyMCA8bWVA and rev_span: q0qLeBxj553tAM2cjZCqxPWIVJwunVDEdpzlUroFXXmgOI7A9myk9CqagqNFw1i
Length Problem at pandas_id: 2081 and json item nr: 1295
Length Problem at pandas_id: 1722 and json item nr: 1641
Detail Problem at pandas_id: 1722 and json item nr: 1641
Orig_flat_list: - - and rev_span: -
Length Problem at pandas_id: 3637 and json item nr: 1673
Detail Problem at pandas_id: 3637 and json item nr: 1673
Orig_flat_list: q0qLeBxj553tAM2cjZCqxPWIVJwu

In [12]:
df_shuffled_new=df_shuffled_new[df_shuffled_new['new_doc'] != ""]
df_shuffled_new.shape

(2491, 8)

# Create CONLL Data

In [13]:
final_df = pd.concat([df_shuffled_old, df_shuffled_new])
final_df.reset_index(inplace=True)
final_df.shape

(3507, 9)

In [14]:
final_df['conll'] = final_df["new_doc"].apply(lambda doc: CoNLL.convert_dict(doc.to_dict() ))

# Store or load dataset

In [15]:
del final_df['doc']
del final_df['vendor']
final_df.rename(columns={"new_doc": "stanza_doc"}, inplace=True)
#final_df.to_pickle('intermediate_store/final_df_v1.0.pickle')
final_df.columns

Index(['index', 'dream_id', 'name', 'description', 'lang', 'clean_description',
       'stanza_doc', 'conll'],
      dtype='object')

In [16]:
final_df = pd.read_pickle('intermediate_store/final_df_v1.0.pickle')

# Train Test Split

In [17]:
train, test = train_test_split(final_df, test_size=0.2, random_state=42)
train, dev = train_test_split(train, test_size=0.2, random_state=42)
print('Train: %d lines, dev: %d lines, test: %d lines'%(len(train), len(dev), len(test)))

Train: 2244 lines, dev: 561 lines, test: 702 lines


In [18]:
final_df.columns

Index(['index', 'dream_id', 'name', 'description', 'lang', 'clean_description',
       'stanza_doc', 'conll'],
      dtype='object')

# Write to FILE

In [19]:


def write_conll_to_file(df, filename, with_comments, full_conll_format):
    
    with io.open(filename, 'w', encoding='utf8') as fout:
    
        for i, row in df.iterrows():
            
            if with_comments:
                fout.write('#NAME\t' + row['name'] +'\n')
                fout.write('#original_description\t' + row['description'] + '\n')
                fout.write('#dream_id\t' + str(row['dream_id']) + '\n')
                
            for sentence in row['conll']:
                for line in sentence:
                    
                    if full_conll_format: #write all CONLL columns 
                        for j, token in enumerate(line):
                            
                            if j >0:
                                 fout.write('\t')
                            fout.write(token)
                        fout.write('\n')
                    else:#only write token and label
                        label = line[9].split('|')[2].split('=')[1] # line[9] looks like: 'start_char=0|end_char=4|label=O'
                        fout.write('{token}\t{tag}\n'.format(token=line[1], tag=label))
                #Write '.' so I can separate Sentences for long texts.
                fout.write('.\tO\n')
                
            #After each item we need an empty line as separator         
            fout.write('\n')


In [20]:
output_dir = 'reviewed_data/training_files/'
train_file_path= output_dir + 'train.txt'
dev_file_path= output_dir + 'dev.txt'
test_file_path= output_dir + 'test.txt'


comments = False
full_conll=False

write_conll_to_file(train, train_file_path, comments, full_conll)
write_conll_to_file(dev, dev_file_path, comments, full_conll)
write_conll_to_file(test, test_file_path, comments, full_conll)


In [21]:
#final_df['conll'][0]

In [22]:
# Write Dataframes as json
json_train_file_path= output_dir + 'train.json'
json_dev_file_path= output_dir + 'dev.json'
json_test_file_path= output_dir + 'test.json'


train[['index', 'dream_id', 'name', 'description', 'lang', 'clean_description', 'conll']].to_json(json_train_file_path, orient="records")
dev[['index', 'dream_id', 'name', 'description', 'lang', 'clean_description', 'conll']].to_json(json_dev_file_path, orient="records")
test[['index', 'dream_id', 'name', 'description', 'lang', 'clean_description', 'conll']].to_json(json_test_file_path, orient="records")
